In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
import json

In [3]:
sc = SparkContext('local[8]', 'task2')

In [4]:
reviews_txt = sc.textFile('asnlib/publicdata/review.json')

In [5]:
reviews_json = reviews_txt.map(lambda x: json.loads(x))

In [6]:
reviews_json.first()

{'review_id': '-I5umRTkhw15RqpKMl_o1Q',
 'user_id': '-mA3-1mN4JIEkqOtdbNXCQ',
 'business_id': 'mRUVMJkUGxrByzMQ2MuOpA',
 'stars': 1.0,
 'text': "Walked in around 4 on a Friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. Don't even think they realized we walked in. However everyone at the bar noticed we walked in!!! Service was non existent at best. Not a good way for a new business to start out. Oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. SMDH!!!",
 'date': '2017-12-15 23:27:08'}

In [ ]:
business_review_count = reviews_json.map(lambda x: (x['business_id'], 1)).reduceByKey(lambda a,b: a + b)

In [ ]:
business_review_count.first()

In [ ]:
business_review_count.take(20)

In [ ]:
reviews_json.getNumPartitions()

In [ ]:
business_review_count.getNumPartitions()

In [ ]:
filtered_business_review_count = business_review_count.filter(lambda x: x[1]>50)

In [ ]:
results = {}
results['default'] = {}
results['default']['n_parititions'] = reviews_json.getNumPartitions()
results['default']['n_items'] = reviews_json.glom().map(len).collect()
results['default']['result'] = filtered_business_review_count.map(lambda x: list(x)).collect()

In [ ]:
results

In [ ]:
business_ids = list(reviews_json.map(lambda x: x['business_id']).distinct().collect())

In [ ]:
def partition_business_id(id):
    return business_ids.index(id)

In [ ]:
# reviews_custom_json = reviews_txt.map(lambda x: json.loads(x)).map(lambda x: (x['business_id'], x)).partitionBy(32, partition_business_id)

In [ ]:
# reviews_custom_json.first()

In [ ]:
n_partitions = 8

In [ ]:
def hash_partition_business_id(id):
    return sum([ord(x) for x in id[:4]]) % n_partitions

In [ ]:
def hash_partition_business_id(id):
    return sum([ord(x) for x in id])

In [ ]:
reviews_custom_new_json = reviews_json.map(lambda x: (x['business_id'], x)).partitionBy(n_parititions, hash_partition_business_id)

In [ ]:
reviews_custom_new_json.first()

In [ ]:
reviews_custom_new_json.map(lambda x: (x[0], 1)).reduceByKey(lambda a,b: a + b).take(40

In [7]:
with open('asnlib/publicdata/stopwords') as f:
    f_stopwords = f.read()
    f.close()


In [8]:
stopwords_list = [x.strip().lower() for x in f_stopwords.split()]

In [28]:
stopwords_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 '

In [11]:
import re

In [24]:
words_count_rdd = reviews_json.map(lambda x: (x['text'])).flatMap(lambda line: line.lower().split()).map(lambda x: re.sub(r'[\s\(\[,\.\!\?\:\;\]\)]', '', x)).map(lambda x: if x is not  (x.strip(), 1)).filter(lambda x: x[0] not in stopwords_list and x[0]).reduceByKey(lambda a, b: a + b)

In [25]:
words_count_rdd.take(10)

[('even', 222066),
 ('noticed', 19055),
 ('well', 193783),
 ('', 179894),
 ('ever', 115442),
 ('sure', 115419),
 ('fresh', 103964),
 ('pittsburgh', 8474),
 ('poland', 53),
 ('part', 43356),
 ('sell', 12298),
 ('buy', 30176),
 ('prices', 75164),
 ('wait', 120521),
 ('still', 116205),
 ('almost', 48419),
 ('flip', 1387),
 ('know', 135225),
 ('elna', 2),
 ('took', 109473),
 ('waitress', 44386),
 ('card/', 7),
 ('away', 69825),
 ('toss', 911),
 ('phone', 40187),
 ("let's", 8500),
 ('fits', 1656),
 ('literally', 21337),
 ('handle', 6476),
 ('truth', 1864),
 ('gotten', 12463),
 ('department', 6797),
 ('seen', 28110),
 ('places', 61275),
 ('kids', 38204),
 ('living', 10176),
 ('whatsoever', 2521),
 ('kabob', 1570),
 ('sold', 8504),
 ('absolutely', 45255),
 ('hopefully', 6538),
 ('willing', 10268),
 ('stay', 42692),
 ('customize', 1489),
 ('combo', 12665),
 ('posted', 4967),
 ('negative', 11103),
 ('crab', 22797),
 ('began', 5073),
 ('ordering', 23400),
 ('"going', 77),
 ('margeritas', 9),
 ('

In [26]:
# 1.E

words_count_rdd.sortBy(lambda x: x[1],ascending=False).map(lambda x: x[0]).take(20)


['food',
 'place',
 'good',
 'great',
 'service',
 'like',
 'time',
 'get',
 'one',
 'would',
 'back',
 "it's",
 'go',
 'really',
 'also',
 'us',
 'got',
 'even',
 '-',
 'nice']

In [9]:
words_count_rdd = reviews_json.map(lambda x: (x['text'])).flatMap(lambda line: line.lower().split()).map(lambda x: (x.strip(' ([,.!?:;])'), 1)).reduceByKey(lambda a, b: a + b).filter(lambda x: x[0] not in stopwords_list and x[0] is not '')

In [10]:
words_count_rdd.take(10)

[('even', 222065),
 ('noticed', 19055),
 ('well', 193783),
 ('ever', 115440),
 ('sure', 115419),
 ('fresh', 103963),
 ('pittsburgh', 8474),
 ('poland', 53),
 ('part', 43356),
 ('sell', 12298)]

In [11]:
words_count_rdd.sortBy(lambda x: x[1],ascending=False).map(lambda x: x[0]).take(40)

['food',
 'place',
 'good',
 'great',
 'service',
 'like',
 'time',
 'get',
 'one',
 'would',
 'back',
 "it's",
 'go',
 'really',
 'also',
 'us',
 'got',
 'even',
 '-',
 'nice',
 "don't",
 'best',
 'well',
 "i'm",
 'came',
 'always',
 'ordered',
 "i've",
 'staff',
 'love',
 "didn't",
 'first',
 'order',
 'never',
 'went',
 'friendly',
 'restaurant',
 'come',
 'try',
 'could']